In [1]:
import os
import numpy as np
import pandas as pd

#images
import cv2

#modeling
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

#visualizations
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
tf.config.get_visible_devices()

2021-12-25 22:30:43.564282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:43.646315: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:43.647052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
if 'GPU' in str(device_lib.list_local_devices()):
    config = tf.compat.v1.ConfigProto(device_count = {'GPU': 0})
    sess = tf.compat.v1.Session(config=config) 

2021-12-25 22:30:43.697505: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-25 22:30:43.698417: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:43.699139: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:43.699806: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [4]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [5]:
def train_id_to_path(x):
    return '../input/petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '../input/petfinder-pawpularity-score/test/' + x + ".jpg"

#Read in the data and drop unnecessary columns
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

#Add the .jpg extensions to the image file name ids
train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [6]:
#binning columns to test models
train['two_bin_pawp'] = pd.qcut(train['Pawpularity'], q=2, labels=False)
train = train.astype({"two_bin_pawp": str})

train['four_bin_pawp'] = pd.qcut(train['Pawpularity'], q=4, labels=False)
train = train.astype({"four_bin_pawp": str})

train['ten_bin_pawp'] = pd.qcut(train['Pawpularity'], q=10, labels=False)
train = train.astype({"ten_bin_pawp": str})

In [7]:
train2bin_stats = train.groupby('two_bin_pawp')
train2bin_stats.describe()

Pawpularity                                                     
                   count       mean        std   min   25%   50%   75%    max
two_bin_pawp                                                                 
0                 5038.0  23.351727   7.628728   1.0  20.0  25.0  29.0   33.0
1                 4874.0  53.220558  18.673900  34.0  39.0  47.0  62.0  100.0

In [8]:
train.head()

,Id,Pawpularity,img_path,two_bin_pawp,four_bin_pawp,ten_bin_pawp
0,0007de18844b0dbbb5e1f607da0606e0,63,../input/petfinder-pawpularity-score/train/000...,1,3,8
1,0009c66b9439883ba2750fb825e1d7db,42,../input/petfinder-pawpularity-score/train/000...,1,2,6
2,0013fd999caf9a3efe1352ca1b0d937e,28,../input/petfinder-pawpularity-score/train/001...,0,1,3
3,0018df346ac9c1d8413cfcc888ca8246,15,../input/petfinder-pawpularity-score/train/001...,0,0,0
4,001dc955e10590d3ca4673f034feeef2,72,../input/petfinder-pawpularity-score/train/001...,1,3,9


In [9]:
test.head()

,Id,img_path
0,4128bae22183829d2b5fea10effdb0c3,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,../input/petfinder-pawpularity-score/test/8f49...


In [10]:
image_height = 128
image_width = 128

#define a function that accepts an image url and outputs an eager tensor
def path_to_eagertensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    #image = tf.image.resize_with_pad(image, image_height, image_width) #optional with padding to retain original dimensions
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [11]:
X = []
for img in train['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X.append(new_img_tensor)
    
print(type(X),len(X))
X = np.array(X)
print(type(X),X.shape)

2021-12-25 22:30:45.962087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:45.963030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:45.963727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:45.964467: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-25 22:30:45.965100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

<class 'list'> 9912
<class 'numpy.ndarray'> (9912, 128, 128, 3)


In [12]:
X_submission = []
for img in test['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X_submission.append(new_img_tensor)
    
print(type(X_submission),len(X_submission))
X_submission = np.array(X_submission)
print(type(X_submission),X_submission.shape)

<class 'list'> 8
<class 'numpy.ndarray'> (8, 128, 128, 3)


In [13]:
y = train['Pawpularity']
print(type(y))

<class 'pandas.core.series.Series'>


In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(8920, 128, 128, 3) (992, 128, 128, 3) (8920,) (992,)


In [15]:
#define the inputs to your model. Basically the shape of the incoming data
inputs = tf.keras.Input(shape=(128,128,3))

#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 61, 61, 16)        2368      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        4640      
_________________________________________________________________
batch_normalization (BatchNo (None, 61, 61, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 31, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 32)        0     

In [17]:
model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [18]:
data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

In [19]:
history = model.fit(
    data_augmentation.flow(x_train,y_train,batch_size=32),
    validation_data = (x_test,y_test),
    steps_per_epoch = len(x_train) // 32,
    epochs = 60
)

2021-12-25 22:32:08.998933: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/60


2021-12-25 22:32:10.977299: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


278/278 [==============================] - 40s 117ms/step - loss: 507.6427 - rmse: 22.5254 - mae: 16.9145 - mape: 81.7122 - val_loss: 487.2693 - val_rmse: 22.0684 - val_mae: 15.4707 - val_mape: 62.5612
Epoch 2/60
278/278 [==============================] - 32s 116ms/step - loss: 476.0444 - rmse: 21.8125 - mae: 16.3472 - mape: 80.9022 - val_loss: 460.6436 - val_rmse: 21.4565 - val_mae: 15.5409 - val_mape: 71.3958
Epoch 3/60
278/278 [==============================] - 32s 115ms/step - loss: 461.5290 - rmse: 21.4771 - mae: 16.1173 - mape: 78.7447 - val_loss: 433.8052 - val_rmse: 20.8216 - val_mae: 15.5327 - val_mape: 73.8190
Epoch 4/60
278/278 [==============================] - 32s 116ms/step - loss: 464.2276 - rmse: 21.5397 - mae: 16.0979 - mape: 79.1861 - val_loss: 442.6101 - val_rmse: 21.0318 - val_mae: 15.6584 - val_mape: 75.3187
Epoch 5/60
278/278 [==============================] - 32s 116ms/step - loss: 461.4278 - rmse: 21.4745 - mae: 16.0700 - mape: 79.1023 - val_loss: 449.3252 - val

In [20]:
cnn_pred = model.predict(X_submission)
print(X_submission.shape, type(X_submission))
print(cnn_pred.shape, type(cnn_pred))

(8, 128, 128, 3) <class 'numpy.ndarray'>
(8, 1) <class 'numpy.ndarray'>


In [21]:
cnn = pd.DataFrame()
cnn['Id'] = test['Id']
cnn['Pawpularity'] = cnn_pred
cnn.to_csv('submission.csv',index=False)

In [22]:
cnn.head(10)

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,40.109463
1,43a2262d7738e3d420d453815151079e,40.212399
2,4e429cead1848a298432a0acad014c9d,40.639473
3,80bc3ccafcc51b66303c2c263aa38486,40.243393
4,8f49844c382931444e68dffbe20228f4,40.645004
5,b03f7041962238a7c9d6537e22f9b017,40.461594
6,c978013571258ed6d4637f6e8cc9d6a3,41.081436
7,e0de453c1bffc20c22b072b34b54e50f,40.168221


In [23]:
cnn.to_csv('/kaggle/working/submission.csv',index=False)